In [ ]:
#quantidade de CNES's localizado na base 
print("CNES's executante na base: ",df_cnes_usuario.count())

In [ ]:
#localizando informações com a base do CNES
df_cnes_usuario = pd.merge(df_cnes_usuario,df_cnes, left_on='CO_CNES', right_on='CO_CNES', how='left')

In [ ]:
#verificar CNES localizado (limpeza)
df_cnes_usuario.dropna(inplace=True)

In [ ]:
#quantidade de CNES localizado na base e tratado
print("Cep's unicos de usuarios: ",df_cnes_usuario.count())

In [ ]:
df_cnes_usuario.head(5)

In [ ]:
df_original_cc = pd.merge(df_original_c, df_cnes_usuario, how="inner", on="CO_CNES")
df_original_cc.head()

In [ ]:
## Calculando distancia (usuario x unidade executante)

In [ ]:
#tratamento nos dados
df_original_cc = df_original_cc.astype({
    'LATITUDE_USUARIO':float,
    'LONGITUDE_USUARIO':float,
    'NU_LONGITUDE':float,
    'NU_LATITUDE':float,
})

In [ ]:
#verificar dados (limpeza)
df_original_cc.dropna(inplace=True)
df_original_cc.info()

In [ ]:
df_original_cc.head(10)

In [ ]:
#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_original_cc.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.NU_LATITUDE, row.NU_LONGITUDE)

    df_original_cc.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeEXE).km, 2)

In [ ]:
#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_tratado.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.LATITUDE_UNIDADEEXE, row.LONGITUDE_UNIDADEEXE

    df_tratado.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeSOL).km, 2)

#quantidade de CNES's localizado na base 
df_cnes_usuario.info()

#Consultar endereço com a API (CEP) (opcional)
for idx, row in df_ceps_usuario.iterrows():
    
    cep = row.CEP_USUARIO
    url = 'https://viacep.com.br/ws/%s/json/' % cep
    headers = {'User-Agent': 'Autociencia/1.0'}
    requisicao = urllib.request.Request(url, headers=headers, method='GET')
    usuario = urllib.request.urlopen(requisicao)
    conteudo = usuario.read().decode('utf-8')
    endereco = json.loads(conteudo)
    #time.sleep(1)
    usuario.close()    
    print(cep)

    if "cep" in endereco:
        df_ceps_usuario.at[idx, 'LOGRADOURO_USUARIO'] = endereco ['logradouro']
        df_ceps_usuario.at[idx, 'BAIRRO_USUARIO'] = endereco ['bairro']
        df_ceps_usuario.at[idx, 'CIDADE_USUARIO'] = endereco ['localidade']
        df_ceps_usuario.at[idx, 'ESTADO_USUARIO'] = endereco ['uf']

    else:
        
        df_ceps_usuario.at[idx, 'LOGRADOURO_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'BAIRRO_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'CIDADE_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'ESTADO_USUARIO'] = 'Não Localizado'

#Criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario ["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO_USUARIO']+ "-" + df_ceps_usuario['BAIRRO_USUARIO'] + "-" +df_ceps_usuario['CIDADE_USUARIO']+ "-" +df_ceps_usuario['ESTADO_USUARIO']
df_ceps_usuario.head()

In [ ]:
#localizando lat e log 
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head()

In [ ]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario

In [ ]:
df_tratado = pd.merge(df_original, df_ceps_usuario, how="inner", on="CEP_USUARIO")
df_tratado.head()

In [ ]:
#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_tratado.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.LATITUDE_UNIDADEEXE, row.LONGITUDE_UNIDADEEXE

    df_tratado.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeSOL).km, 2)

In [ ]:
# https://www.youtube.com/watch?v=-9YbAz6ACmU

In [ ]:




conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\colocar_aqui_o_endereco_da_pasta\nome_do_banco_access.accdb;')
# abrindo conexão com access 

cursor = conn.cursor()
cursor.execute('select PESQUISA, id_cnpj from cns_Enderecos_empresas where result_lat_long is null') # fazendo select para abrir uma tabela

tabela = cursor.fetchall()

url = "https://developers-dot-devsite-v2-prod.appspot.com/maps/documentation/utils/geocoder#q%3D"
print(url)
driver = webdriver.Chrome(executable_path='./chromedriver.exe') #chromedriver.exe
driver.get(url)
time.sleep(5)

for endereco in tabela:    
    element = driver.find_element_by_id("query-input")
    element.clear()
    element.send_keys(endereco.PESQUISA) 
    driver.find_element_by_id("geocode-button").click() #clicando no botão para pesquisqar
    time.sleep(2)

    
    id_cnpj1 = endereco.id_cnpj
    element = driver.find_element_by_id("result-0") 
    element2 = driver.find_element_by_id("result-0") 
    element = element.find_element_by_class_name("result-location")
    element2 = element2.find_element_by_class_name("result-formatted-address")
    lat_long =  element.text
    endereco_econtrado = element2.text
    endereco_econtrado = endereco_econtrado.replace("'","¹")

    lat_long = lat_long.replace(" (type: ROOFTOP)","") #substituindo palavras
    lat_long = lat_long.replace("Location: ","")  #substituindo palavras
    lat_long = lat_long.replace(" (type: GEOMETRIC_CENTER)","")  #substituindo palavras
    lat_long = lat_long.replace(" (type: RANGE_INTERPOLATED)","")  #substituindo palavras
    endereco = endereco.PESQUISA

    print (endereco)     
    print(lat_long, " - ", endereco_econtrado)
    
    sql = """\
        UPDATE tb_Dcnpj
        SET result_lat_long ='{lat_long}',
        endereco_encontrado ='{endereco_econtrado}'
        WHERE id_cnpj ={id_cnpj1}
        """.format(lat_long=lat_long,id_cnpj1=id_cnpj1,endereco_econtrado=endereco_econtrado) #montando sql para inserir lat e long na base

    cursor.execute(sql) #executando sql
    conn.commit()
    print('-----')
    print('-----')


driver.quit()
cursor.quit()
conn.quit()





























In [ ]:
https://www.youtube.com/watch?v=lK8ANM7VkNU 

In [ ]:
https://www.youtube.com/watch?v=gtymrMMRNTY&t=279s 

In [ ]:
https://www.youtube.com/watch?v=LIfscd9SNLo&t=627s

In [ ]:
https://www.youtube.com/watch?v=IKIA_e6aJsw